In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, hour, broadcast

In [ ]:
spark = SparkSession.builder.appName("Pipeline batch").getOrCreate()

In [ ]:
dataframe_industrial = spark.read.csv("./data/industrial_monitoring_logs.csv", header=True, inferSchema=True)

In [ ]:
dataframe_hour_equipment = dataframe_industrial.groupBy(hour(col("timestamp")).alias("hour"), col("equipment")).count().alias("log_count")

In [ ]:
dataframe_metadata = spark.read.csv("./data/equipment_metadata.csv", header=True, inferSchema=True)
dataframe_joined = dataframe_hour_equipment.join(broadcast(dataframe_metadata), dataframe_hour_equipment.equipment == dataframe_metadata.equipment_type)

In [ ]:
os.makedirs("./data/output/", exist_ok=True)
dataframe_joined.write.mode("overwrite").parquet("./data/output/industrial_hourly_equipment_logs.parquet")

In [ ]:
spark.stop()